In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 11.683 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.042 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.371 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT c2, SUM (v1) AS suma
    FROM
        tbl0 LATERAL VIEW explode(c6) tbl10 AS k1,v1
    GROUP BY c2;

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.116 seconds
CREATE TABLE salida
AS
    SELECT c2, SUM (v1) AS suma
    FROM
        tbl0 LATERAL VIEW explode(c6) tbl10 AS k1,v1
    GROUP BY c2;
Query ID = root_20200119222740_1ae27a5e-0fad-42ab-8b98-269a3472e063
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579469532371_0005, Tracking URL = http://92c98434d4d4:8088/proxy/application_1579469532371_0005/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579469532371_0005
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 22:27:48,563 Stage-1 map = 0%,  reduce = 0%
2020-01-19 2

In [6]:
%%hive
--SELECT * 
--FROM salida;

SELECT * 
FROM salida;
OK
A	20
B	200
C	130
D	220
Time taken: 2.07 seconds, Fetched: 4 row(s)


In [7]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200119223027_e5c4d245-eb9d-4003-b7f7-f5f2d60096a4
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579469532371_0006, Tracking URL = http://92c98434d4d4:8088/proxy/application_1579469532371_0006/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579469532371_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-19 22:30:34,117 Stage-1 map = 0%,  reduce = 0%
2020-01-19 22:30:38,536 Stage-1 map = 100%,  reduce = 0%
MapReduce Total cumulative CPU time: 1 seconds 270 msec
Ended Job = job_1579469532371_0006
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000_0/.hive-staging_hive_2020

In [8]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [9]:
%hive_quit